<a href="https://colab.research.google.com/github/adamgerstenfeld/EcoAction/blob/main/Data_Ingestion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><h1 style ="color:green;">Bed Availability Data - Hypothermia Season</h1>
<center><h3 style ="color:green;">Author: Tauseef Ahmad</h3>    

Import Packages

In [ ]:
import pandas as pd
import os, re

Mount google drive

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Year 2021
Create the output dataframe as per the requirements for year 2021

In [ ]:
# create a empty dataframe which will hold the final data
df_final = pd.DataFrame()
for root, _, files in os.walk("/content/drive/My Drive/2021"):
    for file_name in files:
      # read only xlsx files and files that start with a number
      if (file_name.partition('.')[2] == 'xlsx') & (re.match('[0-9]', file_name.partition('.')[0]) is not None):
        df = pd.DataFrame()
        final_df = pd.DataFrame()
        print (os.path.join(root, file_name))
        # read the excel file
        df = pd.read_excel(os.path.join(root, file_name),header=[8])
        # rename columns
        df.rename(columns={'Unnamed: 0':'Provider Name',
                   'Unnamed: 1':'Phone Number',
                   'Unnamed: 2':'Ward',
                   'Unnamed: 3':'Bed Availability',
                   'Men Served':'Clients Served'},inplace = True)
        # keep only the relevant columns
        df=df[['Provider Name','Phone Number','Ward','Bed Availability','Clients Served','Vacancies']].copy()
        # separate women and men data
        women_df = df[df[df['Provider Name']=='WOMEN'].index[0]+2:df[df.isnull().all(axis=1)].index[0]].copy()
        women_df['Gender Served'] ='Female'
        men_df = df[:df[df['Provider Name']=='WOMEN'].index[0]].copy()
        men_df['Gender Served'] = 'Male'
        #append both men and women dataframes
        final_df = men_df.append(women_df).reset_index(drop=True)
        final_df['Date'] = file_name.partition('.')[0]
        df_final = df_final.append(final_df)

/content/drive/My Drive/2021/June/6-17-21.xlsx
/content/drive/My Drive/2021/June/6-14-21.xlsx
/content/drive/My Drive/2021/June/6-1-21.xlsx
/content/drive/My Drive/2021/June/6-20-21.xlsx
/content/drive/My Drive/2021/June/6-12-21.xlsx
/content/drive/My Drive/2021/June/6-11-21.xlsx
/content/drive/My Drive/2021/June/6-13-21.xlsx
/content/drive/My Drive/2021/June/6-19-21.xlsx
/content/drive/My Drive/2021/June/6-15-21.xlsx
/content/drive/My Drive/2021/June/6-18-21.xlsx
/content/drive/My Drive/2021/June/6-16-21.xlsx
/content/drive/My Drive/2021/June/6-10-21.xlsx
/content/drive/My Drive/2021/June/6-30-21.xlsx
/content/drive/My Drive/2021/June/6-2-21.xlsx
/content/drive/My Drive/2021/June/6-27-21.xlsx
/content/drive/My Drive/2021/June/6-21-21.xlsx
/content/drive/My Drive/2021/June/6-26-21.xlsx
/content/drive/My Drive/2021/June/6-4-21.xlsx
/content/drive/My Drive/2021/June/6-29-21.xlsx
/content/drive/My Drive/2021/June/6-24-21.xlsx
/content/drive/My Drive/2021/June/6-28-21.xlsx
/content/drive/M

In [ ]:
# add month and year columns
df_final['Month'] = pd.to_datetime(df_final['Date']).dt.month
df_final['Year']= pd.to_datetime(df_final['Date']).dt.year

In [ ]:
df_final.info()
df_final.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4215 entries, 0 to 11
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Provider Name     4215 non-null   object
 1   Phone Number      4215 non-null   object
 2   Ward              4215 non-null   object
 3   Bed Availability  4151 non-null   object
 4   Clients Served    4215 non-null   object
 5   Vacancies         4215 non-null   object
 6   Gender Served     4215 non-null   object
 7   Date              4215 non-null   object
 8   Month             4215 non-null   int64 
 9   Year              4215 non-null   int64 
dtypes: int64(2), object(8)
memory usage: 362.2+ KB


,Provider Name,Phone Number,Ward,Bed Availability,Clients Served,Vacancies,Gender Served,Date,Month,Year
0,801 East\n(Catholic Charities),561-4014,8,240,235,5,Male,6-17-21,6,2021
1,Adams Place Shelter\n(Catholic Charities),832-8317,5,115,115,0,Male,6-17-21,6,2021
2,New York Avenue HAC\n(Catholic Charities),832-2359,5,200,200,0,Male,6-17-21,6,2021
3,CCNV-Drop in Center Federal City\n(Community ...,393-1909,6,67,54,13,Male,6-17-21,6,2021
4,Pat Handy Legacy,940-6094,2,118,113,5,Male,6-17-21,6,2021


In [ ]:
mat = re.match('(\d{1,2})[/.-](\d{1,2})[/.-](\d{4})$', '1-04-2021')
mat